In [ ]:
# 아무 파이썬 개발환경이나 받아서 실행하면 됨
# v1.1(2022-10-18) 가상의 차량속도와 통신 주기를 알기 쉬운 형태로 변경

import math

def buildWaypoint(vertexList, velocity, period):
    """입력된 각 꼭지점과 속도를 바탕으로 웨이포인트를 생성한다."""
    """vertexList[]:각 지점의 GPS좌표값 튜플(Lat, Lon)의 리스트"""
    """velocity: 속도(km/h)"""
    """period: 측정주기(s) -> 몇 초마다 점을 찍는가?"""
    
    velocity_mps = velocity/3.6
    
    # "각 구간을 몇 등분할 것인가?" 정보를 담은 리스트
    section_float = []
    for a in range(0, len(vertexList), 1):
        b = 0 if a>=len(vertexList)-1 else a+1
        m = math.pow(vertexList[b][0]-vertexList[a][0], 2)
        n = math.pow(vertexList[b][1]-vertexList[a][1], 2)
        section_float.append( math.sqrt(m+n) * 1e5 / velocity_mps / period )
        section_int = [math.floor(x)+1 for x in section_float]
        
#     print(section_int)
#     print(len(vertexList))
#     print(range(0, len(vertexList)))
#     print(list(range(0, 5)))
    
    # 출력할 웨이포인트 리스트
    waypoint = []
    # 각 직선구간에 대하여,
    for i in range(0,len(vertexList)):
        j = 0 if i>=len(vertexList)-1 else i+1
        # 분할된 새로운 GPS좌표를 계산해서 웨이포인트에 추가한다.
        for q in range(0,section_int[i]):
            myLat = vertexList[i][0] + (vertexList[j][0]-vertexList[i][0])/section_int[i]*q
            myLon = vertexList[i][1] + (vertexList[j][1]-vertexList[i][1])/section_int[i]*q
            waypoint.append((myLat,myLon))
#     print(waypoint)
#     print(len(waypoint))
    
    return waypoint

In [ ]:
import json
import time
import requests
from datetime import datetime

def HttpPost_cold(gpsLat, gpsLon, url:str, coldSerNum:str, timeStamp:str=datetime.now().strftime('%y%m%d%H%M%S')):
    """gpsLat: Latitude"""
    """gpsLon: longitude"""
    """coldSerNum: ex) \"PLZ2022C2\" """
    """timeStamp: ex) \"221017210459\", default: current system time"""
    
    myJson = {
        "data": 
        [
            {"ser_num":coldSerNum,"lat_c":round(gpsLat,6),"lon_c":round(gpsLon,6),
             "dof_vib":3.2,"dof_tilt":1,"err_code":0,"temp_in":8,"temp_out":24,"temp_goal":5,
             "cold_min_left":251,"bat_level":82,"bat_volt":14.23,
             "time_stamp":timeStamp,"wifi_ssid":"plzhwtest","sd_mb_now":84},
        ]
    }
#     print(myJson)
    
    myHeader = {"DeviceID": coldSerNum, "Content-Type": "application/json;charset=UTF-8"}
    x = requests.post(url, json = myJson, headers= myHeader)
    
    print(x.text)
#     print("---------- Cold response: ", x.status_code,"---------」")
    print(x.status_code)
    
    

def HttpPost_vehicle(gpsLat, gpsLon, url:str, vehiclePhoneNum:str, vehicleID:str, timeStamp:str=datetime.now().strftime('%y%m%d%H%M%S')):
    """gpsLat: Latitude"""
    """gpsLon: longitude"""
    """vehiclePhoneNum: ex) \"01001001002\" """
    """coldSerNum: ex) \"PLZ2022V2\"" """
    """timeStamp: ex) \"221017210459\", default: current system time"""
    
    myJson = {
        "data": 
        [
            {"01":1,"11":vehiclePhoneNum,"12":vehicleID,"13":"d47c44400348","16":"abcde12345","17":"354481106012345",
             "18":"AMM574A","19":"AMM574A-10-00-LG","1A":"0612345","21":timeStamp,"32":100,"33":1,"31":55,"34":55,"35":12,
             "36":1,"37":"00","38":1,"41":round(gpsLat,6),"42":round(gpsLon,6),"43":0.798,"44":231.8,"45":"06","46":13.91,"51":1,"52":1,"5E":1,
             "5F":1,"61":-75,"62":-16,"63":13,"64":-50,"65":"0034190E","66":"450","67":"08","68":"0002"},
        ]       
    }
    print(myJson)
    
    myHeader = {"DeviceID": vehicleID, "Content-Type": "application/json;charset=UTF-8"}
    x = requests.post(url, json = myJson, headers= myHeader)
    
    print(x.text)
#     print("---------- Vehicle response: ", x.status_code,"---------」")
    print(x.status_code)
    

    

In [ ]:
# GPS좌표 알아내는 주소 tablog.neocities.org/keywordposition.html

# ykd = [(36.124083,128.181111),(36.12416,128.187743),(36.121289,128.187755),(36.12127,128.183855),(36.121231,128.181195)]
hgd = [(36.116854,128.124157),(36.116672,128.125938),(36.115368,128.125482),(36.115368,128.125279),(36.115018,128.125049),(36.115875,128.124098)]
ykd = [(36.124039661000104, 128.18770755695155),(36.12417240395391, 128.19067529762043),(36.121324186730284, 128.19073224878875),(36.12118265315686, 128.187742268562),(36.12032179229347, 128.1877681754574),(36.12035333712074, 128.18731325925123),(36.12077731261892, 128.18726966144365),(36.12121289089066, 128.1865098216519),(36.1212011691264, 128.18769256497188),(36.12336389527366, 128.18769737643308)]

print(ykd)
print(len(ykd))
print(hgd)
print(len(hgd))

In [ ]:
# # 좌표값 확인용 그래프 출력
# import matplotlib.pyplot as plt
# # %matplotlib inline
# # 그래프를 새 창에서 띄우는 경우
# %matplotlib qt5

# ykd_waypoint = buildWaypoint(ykd, 25.0, 3)
# hgd_waypoint = buildWaypoint(hgd, 25.0, 3)

# # print(ykd_waypoint)
# # print(hgd_waypoint)

# ykdLat = [k[0] for k in ykd_waypoint]
# ykdLon = [k[1] for k in ykd_waypoint]

# hgdLat = [k[0] for k in hgd_waypoint]
# hgdLon = [k[1] for k in hgd_waypoint]

# # print(ykdLat)
# fig = plt.figure(figsize=(12,6))
# ax = [plt.subplot(1,2,x) for x in [1,2]]

# plt.autoscale(enable=True)

# ax[0].scatter(ykdLon,ykdLat)
# ax[1].scatter(hgdLon,hgdLat)

# fig.show()

In [ ]:
# %matplotlib qt5
# import matplotlib.pyplot as plt
# from matplotlib.animation import FuncAnimation
# import random

# ykdLat = [k[0] for k in ykd_waypoint]
# ykdLon = [k[1] for k in ykd_waypoint]

# hgdLat = [k[0] for k in hgd_waypoint]
# hgdLon = [k[1] for k in hgd_waypoint]

# fig = plt.figure(figsize=(12,6))
# ax = plt.subplot(1,2,1)
# plt.autoscale(enable=True)

# ax2 = plt.subplot(1,2,2)

# def myRTplot(i):
#     dataLen = len(ykdLat)
#     color = "#%06x" % random.randint(0, 0xFFFFFF)
#     colorList = [color for x in range(0,dataLen)]
    
#     ax.cla()
#     ax.scatter(ykdLat, ykdLon, c=colorList)

# def myRTplot2(i):
#     dataLen = len(hgdLat)
#     color = "#%06x" % random.randint(0, 0xFFFFFF)
#     colorList = [color for x in range(0,dataLen)]
    
#     ax2.cla()
#     ax2.scatter(hgdLat, hgdLon, c=colorList)


# ani = FuncAnimation(fig, myRTplot, interval=500)
# ani2 = FuncAnimation(fig, myRTplot2, interval=500)
# plt.show()
    
    


In [ ]:
import time

def main():
    velocity = 25.0  # 차량 이동속도(단위: km/h)
    period = 3  # 전송 주기(단위: 초)
    
    ykd_waypoint = buildWaypoint(ykd, velocity=velocity, period=period)
    hgd_waypoint = buildWaypoint(hgd, velocity=velocity, period=period)

    gLen = len(ykd_waypoint)
    hLen = len(hgd_waypoint)
    g=0; h=0

    while(True):
#         g= 0 if g>=gLen-1 else g+1
        g= g+1 if g<gLen-1 else 0
        gLat = ykd_waypoint[g][0]
        gLon = ykd_waypoint[g][1]

        print("「---------","no2(율곡동)",g,"/",gLen,"----------")
        HttpPost_cold(gLat, gLon, "http://gps.plzlab.com/coldChain/insert", "PLZ2022C2")
        HttpPost_vehicle(ykd_waypoint[g][0], ykd_waypoint[g][1], "http://gps.plzlab.com/pmIot/devices/SUL01", "01001001002", "PLZ2022V2")
#         HttpPost_cold(gLat, gLon, "http://gps.plzlab.com/coldChain/insert", "PLZ2022C4")
#         HttpPost_vehicle(ykd_waypoint[g][0], ykd_waypoint[g][1], "http://gps.plzlab.com/pmIot/devices/SUL01", "01001001004", "PLZ2022V4")
        
        
        h= 0 if h>=hLen-1 else h+1
        hLat = hgd_waypoint[h][0]
        hLon = hgd_waypoint[h][1]

        print("「---------","no3(황금동)",h,"/",hLen,"----------")
        HttpPost_cold(hLat, hLon, "http://gps.plzlab.com/coldChain/insert", "PLZ2022C3")
        HttpPost_vehicle(hgd_waypoint[h][0], hgd_waypoint[h][1], "http://gps.plzlab.com/pmIot/devices/SUL01", "01001001003", "PLZ2022V3")
#         HttpPost_cold(hLat, hLon, "http://gps.plzlab.com/coldChain/insert", "PLZ2022C5")
#         HttpPost_vehicle(hgd_waypoint[h][0], hgd_waypoint[h][1], "http://gps.plzlab.com/pmIot/devices/SUL01", "01001001005", "PLZ2022V5")

        time.sleep(period)
        
main()